In [24]:
import requests
import pandas as pd
import re
from datetime import datetime, timedelta

In [6]:
# 1. CSV 파일 불러오기
forecast_df = pd.read_csv("./forecast_agg_test.csv")
forecast_df["일시"] = pd.to_datetime(forecast_df["일시"])

# 각 지점별 최신 날짜 찾기
latest_dates = forecast_df.groupby("지점")["일시"].max()

# 결과 출력
print(latest_dates)

지점
22105   2025-01-12
22107   2025-01-12
22186   2025-01-12
22188   2025-01-12
22189   2025-01-12
22190   2025-01-12
22193   2020-07-17
22297   2025-01-12
22302   2025-01-12
22305   2024-11-23
Name: 일시, dtype: datetime64[ns]


In [72]:
# API 요청
url = "https://apihub.kma.go.kr/api/typ01/url/sea_obs.php"
params = {
    "tm": "20210224",  # 예제 날짜 (가장 최근 날짜로 변경 가능)
    "stn": "22305",  # 전체 지점 데이터 요청
    "authKey": "ZnBKZOsfRBmwSmTrH0QZIA",  # 실제 사용 가능한 인증키 필요
    "help": "1"  # 도움말 제거 (데이터만 가져오기)
}
response = requests.get(url, params=params)
print(response.text)

# # 정규식 패턴으로 데이터 추출
# pattern = r"B,\s*(\d+),\s*(\d+),\s*[\w가-힣]+,\s*([\d.]+),\s*([\d.]+),\s*([\d.]+),\s*([\d.]+),\s*([\d.]+),\s*([\d.]+),\s*([\d.]+),\s*([\d.]+),\s*([\d.]+),\s*([\d.-]+)"
# match = re.search(pattern, response.text)

# data = {
#     "TM": match.group(1),
#     "STN": match.group(2),
#     "LON": match.group(3),
#     "LAT": match.group(4),
#     "WH": match.group(5),
#     "WD": match.group(6),
#     "WS": match.group(7),
#     "WS_GST": match.group(8),
#     "TW": match.group(9),
#     "TA": match.group(10),
#     "PA": match.group(11),
#     "HM": match.group(12),
# }
    
# df = pd.DataFrame([data])
# df

#START7777
#--------------------------------------------------------------------------------------------------
#  해상 관측자료 [입력인수형태][예] ?tm=201001051200&stn=22101&help=1
#                      요청한 시간에서 -59분 ~ 00분 사이의 자료를 모두 표출
#                      stn=0 이거나 없으면, 전체 지점 표출
#--------------------------------------------------------------------------------------------------
#  1. TP      : 관측종류 (B:BUOY, C:파고BUOY, D:표류BUOY, L:등표, N:조위관측소, F:연안방재, G:파랑계, J:기상1호)
#  2. STN_ID  : 지점 ID
#  3. STN_KO  : 지점명
#  4. TM      : 관측시각(KST, 년월일시분)
#  5. WH      : 유의파고 (m) 
#  6. WD      : 풍향 (degree)
#  7. WS      : 풍속 (m/s)
#  8. WS_GST  : GUST 풍속 (m/s)
#  9. TW      : 해수면 온도 (C)
# 10. TA      : 기온 (C)
# 11. PA      : 해면기압 (hPa)
# 12. HM      : 상대습도 (%)
#--------------------------------------------------------------------------------------------------
#            TM       STN                   STN           LON           LAT     WH   WD     WS     WS     TW     TA      PA     HM
#           KST        

In [71]:
# API URL & 공통 파라미터
url = "https://apihub.kma.go.kr/api/typ01/url/sea_obs.php"
stn = "22302"  # 측정소 ID
auth_key = "ZnBKZOsfRBmwSmTrH0QZIA"  # 실제 인증키 필요

# 측정ID: 


# 날짜 범위 설정 (2025-01-13 00시 ~ 2025-01-30 23시)
start_time = datetime(2025, 1, 13, 0, 0)
end_time = datetime(2025, 1, 30, 23, 0)
time_step = timedelta(hours=1)  # 1시간 단위 증가

# 결과 저장용 리스트
data_list = []

# tm 값을 생성하면서 API 요청 수행
current_time = start_time
while current_time <= end_time:
    tm = current_time.strftime("%Y%m%d%H00")  # YYYYMMDDHH00 형식
    params = {
        "tm": tm,
        "stn": stn,
        "authKey": auth_key,
        "help": "1"
    }
    
    response = requests.get(url, params=params)

    # 정규식 패턴
    pattern = r"B,\s*(\d+),\s*(\d+),\s*[\w가-힣]+,\s*([\d.]+),\s*([\d.]+),\s*([\d.]+),\s*([\d.]+),\s*([\d.]+),\s*([\d.]+),\s*([\d.]+),\s*([\d.]+),\s*([\d.]+),\s*([\d.-]+)"
    match = re.search(pattern, response.text)

    # 기본 데이터 구조 (결측치를 빈 값으로 초기화)
    data = {
        "TM": tm,
        "STN": "",
        "LON": "",
        "LAT": "",
        "WH": "",
        "WD": "",
        "WS": "",
        "WS_GST": "",
        "TW": "",
        "TA": "",
        "PA": "",
        "HM": "",
    }

    # 데이터가 정상적으로 매칭되었을 경우 업데이트
    if match:
        extracted_data = {
            "TM": match.group(1),
            "STN": match.group(2),
            "LON": match.group(3),
            "LAT": match.group(4),
            "WH": match.group(5),
            "WD": match.group(6),
            "WS": match.group(7),
            "WS_GST": match.group(8),
            "TW": match.group(9),
            "TA": match.group(10),
            "PA": match.group(11),
            "HM": match.group(12),
        }
        
        # 빈 데이터 구조를 업데이트
        for key in extracted_data:
            if extracted_data[key]:  # 값이 있으면 업데이트
                data[key] = extracted_data[key]
                
    data_list.append(data)  # 데이터 추가
    current_time += time_step  # 1시간 증가

# 리스트를 데이터프레임으로 변환
df = pd.DataFrame(data_list)
df

# CSV로 저장 (선택)
df.to_csv(f"forecast_0131_{stn}.csv", index=False, encoding="utf-8")

In [ ]:
def create_pivot_table(df, index_cols, columns_col, values_col):
    """
    피벗 테이블 생성 함수.
    Args:
        df (pd.DataFrame): 입력 데이터프레임.
        index_cols (list): 기준 컬럼 리스트 (피벗 테이블의 인덱스 컬럼).
        columns_col (str): 분류 기준 컬럼명 (피벗 테이블의 컬럼).
        values_col (str): 값 컬럼명 (피벗 테이블에 채울 값).
    Returns:
        pd.DataFrame: 생성된 피벗 테이블.
    """
    # 피벗 테이블 생성
    pivot_df = df.pivot_table(
        index=index_cols,
        columns=columns_col,
        values=values_col
    )
    # 다중 컬럼 이름 정리 (예: '광어' -> '광어_avgPrice')
    pivot_df.columns = [f"{col}_{values_col}" for col in pivot_df.columns]
    # 인덱스 초기화
    pivot_df.reset_index(inplace=True)
    
    return pivot_df

In [124]:
# 기존 데이터와 새로운 데이터 병합
merged_df = pd.concat([forecast_df, filtered_api_df], ignore_index=True)

# 중복 제거 (지점, 일시 기준으로 최신 데이터 유지)
merged_df = merged_df.drop_duplicates(subset=["지점", "일시"], keep="last")

# 병합된 데이터 저장
merged_df.to_csv("test_forecast_agg.csv", index=False)

# 최종 데이터 확인
print(merged_df.tail())

          지점                  일시   풍속  GUST풍속    현지기압    습도    기온    수온  최대파고  \
23914  33016 2025-01-31 15:00:00  4.1     4.9   -99.0 -99.0 -99.0 -99.0   NaN   
23915  33008 2025-01-31 15:00:00  0.2     0.5   -99.0 -99.0 -99.0 -99.0   NaN   
23916  33020 2025-01-31 15:00:00  2.9     3.5   -99.0 -99.0 -99.0 -99.0   NaN   
23917  33021 2025-01-31 15:00:00  3.8     4.3   -99.0 -99.0 -99.0 -99.0   NaN   
23918  22003 2025-01-31 15:00:00  1.9     4.5  1023.6  64.0  11.5 -99.0   NaN   

       유의파고  평균파고  파주기   지점명          경도         위도    WH     WD  
23914   NaN   NaN  NaN    울산  129.456300  35.581000 -99.0  161.0  
23915   NaN   NaN  NaN    영덕  129.382740  36.338740 -99.0  123.0  
23916   NaN   NaN  NaN   남항진  128.961190  37.761130 -99.0  128.0  
23917   NaN   NaN  NaN   구룡포  129.549650  35.951830 -99.0  197.0  
23918   NaN   NaN  NaN  기상1호  126.569443  33.235279 -99.0   95.0  
